In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imputed-train-data/imputed_y_train.csv
/kaggle/input/imputed-train-data/imputed_x_train.csv
/kaggle/input/imputation-new-1/imputed_13k.csv
/kaggle/input/imputation-new-1/imputed_11k.csv
/kaggle/input/prml-data/sample_submission.csv
/kaggle/input/prml-data/bikers.csv
/kaggle/input/prml-data/bikers_network.csv
/kaggle/input/prml-data/tour_convoy.csv
/kaggle/input/prml-data/tours.csv
/kaggle/input/prml-data/train.csv
/kaggle/input/prml-data/test.csv


# Creating the submission

In [44]:
import datetime
def conv_to_time(a):
    my_string = a
    b = datetime.datetime.strptime(my_string, '%d-%m-%Y %H:%M:%S')
    c = datetime.datetime.timestamp(b)
    return c

train2 = pd.read_csv('../input/imputation-new-1/imputed_11k.csv')
train3 = pd.read_csv('../input/imputation-new-1/imputed_13k.csv')
x_t = train3.drop(labels=['Unnamed: 0', 'like/dislike'], axis=1)
y_t = train3['like/dislike'].copy()

x_t.timestamp = x_t.timestamp.apply(conv_to_time)
x_t.timestamp = (x_t.timestamp-x_t.timestamp.mean())/x_t.timestamp.std()
x_t.head()

,biker_id,tour_id,invited,timestamp
0,DA44012,RU29072432,0,-0.623708
1,EC36494,QU2418051,0,0.358531
2,EC36494,TR03627753,0,0.358551
3,EC36494,WY9782433,0,0.358530
4,FF74997,YT1853303,0,1.069049


In [27]:
from sklearn.preprocessing import OrdinalEncoder, normalize
from sklearn.model_selection import cross_val_score, GridSearchCV
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.svm import SVC

In [50]:
model = KNC()
enc = OrdinalEncoder()
x_enc_qual = pd.DataFrame(enc.fit_transform(x_t[['biker_id', 'tour_id']]), columns=['biker_id', 'tour_id'])
x_enc = x_t
x_enc.tour_id = x_enc_qual.tour_id
x_enc.biker_id = x_enc_qual.biker_id
basescore = cross_val_score(model, x_enc, y_t).mean()
###
param_grid = {'n_neighbors' : range(5,101,5),'weights':('uniform', 'distance')}
grid = GridSearchCV(model, param_grid)
grid.fit(x_enc, y_t)
print(grid.best_estimator_)
print(grid.best_score_)

KNeighborsClassifier(n_neighbors=95)
0.5261731533681335


In [30]:
model2 = AdaBoostClassifier()
enc = OrdinalEncoder()
x_enc_qual = pd.DataFrame(enc.fit_transform(x_t[['biker_id', 'tour_id']]), columns=['biker_id', 'tour_id'])
x_enc = x_t
x_enc.tour_id = x_enc_qual.tour_id
x_enc.biker_id = x_enc_qual.biker_id
basescore = cross_val_score(model2, x_enc, y_t).mean()
###
param_grid = {'base_estimator' : [LogisticRegression(), SVC(probability='True', kernel='linear'), DecisionTreeClassifier(max_depth=1)],'n_estimators':[50, 100, 150]}
grid = GridSearchCV(model2, param_grid)
grid.fit(x_enc, y_t)
grid.best_estimator_
grid.best_score_

0.8184182121874202

In [ ]:
model3 = SVC()
enc = OrdinalEncoder()
x_enc_qual = pd.DataFrame(enc.fit_transform(x_t[['biker_id', 'tour_id']]), columns=['biker_id', 'tour_id'])
x_enc = x_t
x_enc.tour_id = x_enc_qual.tour_id
x_enc.biker_id = x_enc_qual.biker_id
basescore = cross_val_score(model3, x_enc, y_t).mean()
###
param_grid = {'C':[0.1, 1, 10], 'kernel':['linear', 'rbf', 'poly'], 'class_weight': ['balanced']}
grid = GridSearchCV(model3, param_grid)
grid.fit(x_enc, y_t)
grid.best_estimator_
grid.best_score_

In [11]:
from sklearn.preprocessing import OrdinalEncoder

class submission:
    def __init__(self, xtrain, ytrain, xtest):
        self.train_x = xtrain
        self.train_y = ytrain
        self.test_x = xtest
    
    def create_submission(self,clf):
        classifier = clf
        enc = OrdinalEncoder()
        x_train_enc_cols = pd.DataFrame(enc.fit_transform(self.train_x[['biker_id', 'tour_id']]), columns=['biker_id', 'tour_id'])
        x_train_enc = self.train_x.copy()
        x_train_enc['biker_id'] = x_train_enc_cols['biker_id']
        x_train_enc['tour_id'] = x_train_enc_cols['tour_id']
        classifier.fit(x_train_enc, self.train_y)
        x_enc_cols = pd.DataFrame(enc.fit_transform(self.test_x[['biker_id', 'tour_id']]), columns=['biker_id', 'tour_id'])
        x_enc = self.test_x.copy()
        x_enc['biker_id'] = x_enc_cols['biker_id']
        x_enc['tour_id'] = x_enc_cols['tour_id']
        x = self.test_x.copy()
        ypred = pd.Series(classifier.predict(x_enc),name='ypred')
        ans = pd.concat([x,ypred],axis=1)
        sub = []
        for biker in x.biker_id.drop_duplicates():
            like_tours = ans.tour_id[np.logical_and(ans.biker_id==biker, ans.ypred==1)].to_list()
            dislike_tours = ans.tour_id[np.logical_and(ans.biker_id==biker, ans.ypred==0)].to_list()
            temp = [biker, " ".join(like_tours+dislike_tours)]
            sub.append(temp)
        final_sub = pd.DataFrame(data=sub,columns=['biker_id','tour_id'])
        return final_sub
    
        

In [31]:
subm.head()

,biker_id,tour_id
0,CG33145288,QX16813281 QR69035551 VW3098017 RP07279414 YU2...
1,EC61865653,RR14608095 QS25647788 ST80624055 RW48209109 US...
2,CC23946882,RU97005766 SU77726402 UX1543591 YV890078 RX773...
3,DC82478477,TR67083677 QV19013260 YW217712 SV43004825
4,EC46728549,SP29325544 ST54377311 RU29072432 SR30650403 RR...


In [35]:
clf = grid.best_estimator_
xtrain = x_t
ytrain = y_t
test = pd.read_csv('../input/prml-data/test.csv')
test.timestamp = test.timestamp.apply(conv_to_time)
test.timestamp = (test.timestamp - test.timestamp.mean())/test.timestamp.std()
sub_ = submission(xtrain, ytrain, test)
subm = sub_.create_submission(clf)
subm.to_csv('final_s2.csv', header=True, index=False)
subm.head()

,biker_id,tour_id
0,CG33145288,QX16813281 QR69035551 VW3098017 RP07279414 YU2...
1,EC61865653,RR14608095 QS25647788 ST80624055 RW48209109 US...
2,CC23946882,RU97005766 SU77726402 UX1543591 YV890078 RX773...
3,DC82478477,TR67083677 QV19013260 YW217712 SV43004825
4,EC46728549,SP29325544 ST54377311 RU29072432 SR30650403 RR...
